# 2.1.3: Product benchmarking with sPlot

These trait maps are certainly not the first to be made, and there are several other great attempts at the challenging task of producing global trait maps. This raises an important question for both trait map creators and consumers: which maps should be used and for which purposes?

Here we propose the use of the global vegetation plot dataset sPlot as a benchmark against which existing trait products can be compared, as sPlot is the only global-scale dataset that contains plot-level trait estimates, generally avoiding the biases and pitfalls that come with crowd-sourced species observations.

The trait maps we will be comparing against sPlot are:
- Boonman et al., 2020
- Butler et al., 2017
- Dong et al., 2023
- Schiller et all, 2021
- Madani et al., 2018
- Moreno et al. 2018
- Vallicrosa et al., 2022
- van Bodegom et al., 2014
- Wolf et al., 2022

## Imports and config

In [1]:
import pandas as pd

from src.conf.conf import get_config
from src.conf.environment import log

cfg = get_config()

## Standardize all trait products

Standardize the `all-prods_stacks` set of maps, which includes 0.5$\textdegree$ maps of Leaf N (mass), Leaf N (area), and SLA.

In [20]:
from pathlib import Path
from matplotlib.pylab import f
from src.utils.raster_utils import create_sample_raster, open_raster, xr_to_raster

mapping = {"nit": "X14", "nita": "X50", "sla": "X11"}

src_dir = Path(
    cfg.raw_dir, "other-trait-maps", "all-prods_stacks_sla-nit-nita_05D_2022-02-14"
)

resolutions = [0.5, 1, 2]

for res in resolutions:
    ref_r = create_sample_raster(resolution = res, crs="EPSG:4326")

    for trait, code in mapping.items():
        all_prods_nitm = open_raster(
            Path(src_dir, f"all-prods_{trait}_stack_all-maps_05D_2022-02-14.grd")
        )
        authors = all_prods_nitm.attrs["long_name"]
        names = [n.lower() for n in authors]

        for i, band in enumerate(all_prods_nitm):
            if names[i] == "moreno":
                continue  # We received a separate file for this author

            out_path = Path(
                cfg.interim_dir,
                "other_trait_maps",
                str(res).replace(".", ""),
                f"{code}_{names[i]}.tif",
            )
            out_path.parent.mkdir(parents=True, exist_ok=True)

            log.info(f"Writing {out_path}")
            r = all_prods_nitm.sel(band=i + 1)
            r = r.rio.reproject_match(ref_r)
            r.attrs["long_name"] = names[i]
            xr_to_raster(r, out_path)

2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X14_butler.tif


2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X14_boonman.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X14_schiller.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X14_vallicrosa.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X50_butler.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X50_boonman.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X50_schiller.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X50_vallicrosa.tif
2024-11-12 11:08:14 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X11_bodegom.tif
2024-11-12 11:08:15 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_

Standardize the maps provided from Moreno et al. (2018), which includes 1 km maps of Leaf N (mass), as well as LDMC and Leaf P (mass), which are not used in this comparison.

In [24]:
m = open_raster(
    "/home/dl1070/projects/cit-sci-traits/data/raw/other-trait-maps/AMM_Trait_maps_v3_2023/LNC_1km_v3.tif"
).sel(band=1)

In [27]:
m.values

array([[-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       ...,
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.]], dtype=float32)

In [28]:
import numpy as np


src_path = Path(
    cfg.raw_dir, "other-trait-maps", "AMM_Trait_maps_v3_2023", "LNC_1km_v3.tif"
)

resolutions = [0.01, 0.2, 0.5, 1, 2]
r = open_raster(src_path).sel(band=1)

# The corresponding values should be masked (-2, -1, 100, 0)
r = r.where(r > 0)
r = r.rio.write_nodata(np.nan)

for res in resolutions:
    ref_r = create_sample_raster(resolution = res, crs="EPSG:4326")
    r = r.rio.reproject_match(ref_r)
    out_path = Path(
        cfg.interim_dir,
        "other_trait_maps",
        str(res).replace(".", ""),
        f"X14_moreno.tif",
    )
    out_path.parent.mkdir(parents=True, exist_ok=True)
    log.info(f"Writing {out_path}")
    xr_to_raster(r, out_path)

2024-11-12 11:16:04 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/001/X14_moreno.tif
2024-11-12 11:16:27 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/02/X14_moreno.tif
2024-11-12 11:16:27 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/05/X14_moreno.tif
2024-11-12 11:16:27 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/1/X14_moreno.tif
2024-11-12 11:16:27 UTC - src.conf.environment - INFO - Writing data/interim/other_trait_maps/2/X14_moreno.tif


In [29]:
import xarray as xr

from src.utils.dataset_utils import get_trait_map_fns

def splot_correlation(data: xr.DataArray, trait: str, res: int | float) -> float:
    c = cfg.copy()
    c["model_res"] = str(res).replace(".", "")
    splot_path = [fn for fn in get_trait_map_fns("splot", c) if trait == fn.stem][0]
    print(splot_path)
    # splot = open_raster(splot_path)

    # data = data.rio.reproject_match(splot)

    # return data.corr(splot)

splot_correlation(r, "X14", 0.5)

IndexError: list index out of range